# Memory Hierarchy

## 2 Procedure

### 2.1 Understanding the Program
Without delving into the details of the signal processing application, analyze the flow of the C program.
Observe the data access patterns and identify the critical sequence of accesses which may have a larger
impact on the performance of the system.

### 2.3 Cache L1

#### 2.3.1 Theory of Cache

1. Explain the different types of cache misses: compulsory, capacity, and conflict



2. Explain the different types of cache writing-policies.


### 2.3.2 Cache L1: dimension and block size

a) Consider a memory hierarchy composed of a single cache memory (L1), which interconnects the
SDRAM frame memory and the CPU.\
Considering the characteristics of the available memory devices (see Table 1), and the maximum total cost of the memory hierarchy, determine the maximum
storage space of cache L1.
- NOTES:
    - the size of any of the memory modules (frame buffer, any cache) must be an integer power of 2:
        - L1_size = $2^{MAX}$;
    - do not forget to consider the cost of the 128 kByte frame memory.

In [29]:
from math import pow

BUDGET = 0.02
PRICE_PER_MBYTE_L1 = 10
PRICE_PER_MBYTE_SDRAM = 0.01
SIZE_SDRAM = pow(2,17)

def calculate_price(size_in_bytes, price_per_Mbyte):
    
    price_per_byte = price_per_Mbyte / pow(2,20)

    return price_per_byte * size_in_bytes


sdram_price = calculate_price(SIZE_SDRAM,PRICE_PER_MBYTE_SDRAM)

print("SDRAM price: ", sdram_price)

final_budget = BUDGET - sdram_price

print("Budget antes de L1: " ,final_budget)

i = 0
while (calculate_price(pow(2,i),PRICE_PER_MBYTE_L1) < final_budget):
    i += 1 
i -=1

print("Valor do i é: " , i)



SDRAM price:  0.00125
Budget antes de L1:  0.01875
Valor do i é:  10


b) Consider three different dimensions for the L1 data cache: L1_size $\in$ {$ 2^{MAX}, 2^{MAX−1}, 2^{MAX−2} $}.\
For each of these dimensions, and assuming a direct mapping configuration, use the dineroIV
simulator to evaluate the resulting average data miss-rate considering the following block sizes:
- Block_size $\in$ {$8, 16, 32, 64$}.\
Fill the following table with the obtained data:

$$
\begin{matrix}
& L1\_size = 2^{10} & L1\_size = 2^{9} & L1\_size = 2^{8}\\
Block size = 8\ Bytes & 0.0305 & 0.1247 & 0.1960\\
Block size = 16\ Bytes & 0.0363 & 0.1184 & 0.1829\\
Block size = 32\ Bytes & 0.0770 & 0.1492 & 0.2288\\
Block size = 64\ Bytes & 0.1181 & 0.2021 & 0.3340
\end{matrix}
$$

c) For each L1 cache size, plot the variation of the miss-rate with the size of the block. 

![Graph L1 cache size miss rate with size of the block](./plot1.png#gh-dark-mode-only)

d) By considering the obtained results, select two L1 cache configurations (dimension and block size)
that offer the best trade-off between the cost of the device and the resulting average miss-rate.\
Label in the previous plot the two configurations chosen.

In [30]:
def cost(size_in_bytes, price_per_Mbyte, miss_rate):
    price_per_byte = price_per_Mbyte / pow(2,20)

    return price_per_byte * size_in_bytes * miss_rate

print("Size = 2¹⁰")
print(cost(pow(2,10), PRICE_PER_MBYTE_L1, 0.0305))
print(cost(pow(2,10), PRICE_PER_MBYTE_L1, 0.0363))
print(cost(pow(2,10), PRICE_PER_MBYTE_L1, 0.0770))
print(cost(pow(2,10), PRICE_PER_MBYTE_L1, 0.1181))
print("Size = 2⁹")
print(cost(pow(2,9), PRICE_PER_MBYTE_L1, 0.1247))
print(cost(pow(2,9), PRICE_PER_MBYTE_L1, 0.1184))
print(cost(pow(2,9), PRICE_PER_MBYTE_L1, 0.1492))
print(cost(pow(2,9), PRICE_PER_MBYTE_L1, 0.2021))
print("Size = 2⁸")
print(cost(pow(2,8), PRICE_PER_MBYTE_L1, 0.1960))
print(cost(pow(2,8), PRICE_PER_MBYTE_L1, 0.1829))
print(cost(pow(2,8), PRICE_PER_MBYTE_L1, 0.2288))
print(cost(pow(2,8), PRICE_PER_MBYTE_L1, 0.3340))

Size = 2¹⁰
0.0002978515625
0.0003544921875
0.000751953125
0.0011533203125
Size = 2⁹
0.00060888671875
0.0005781250000000001
0.000728515625
0.00098681640625
Size = 2⁸
0.000478515625
0.00044653320312500004
0.0005585937500000001
0.0008154296875000001


In [31]:
L1_CONFIG_1_COST = cost(pow(2,10), PRICE_PER_MBYTE_L1, 0.0305)
L1_CONFIG_2_COST = cost(pow(2,10), PRICE_PER_MBYTE_L1, 0.0363)

final_price_config_1 = L1_CONFIG_1_COST + calculate_price(SIZE_SDRAM,PRICE_PER_MBYTE_SDRAM)
final_price_config_2 = L1_CONFIG_2_COST + calculate_price(SIZE_SDRAM,PRICE_PER_MBYTE_SDRAM)

print(final_price_config_1)
print(final_price_config_2)

0.0015478515625
0.0016044921875


$$
\begin{matrix}
 & L1\ config 1\\
Cache\ size & 2^{10}\\
Block\ size & 2^{3}\\
Miss\ rate & 0.0305\\
Cost & 0.0015478515625
\end{matrix}
$$

$$
\begin{matrix}
 & L1\ config 2\\
Cache\ size & 2^{10}\\
Block\ size & 2^{4}\\
Miss\ rate & 0.0363\\
Cost & 0.0016044921875
\end{matrix}
$$

#### 2.3.3 Cache L1: set associativity

a) For each of the two L1 cache setups previously selected, evaluate the compulsory, capacity, conflict and total miss-rates when the following configurations are considered:
- set associativity of 1 (direct-mapped), 2, 4, 8.

Fill the following table with the obtained data:

$$
\begin{matrix}
 &  & L1\ config\ 1 &  & \\
Miss\ rate & 1-way & 2-way & 4-way & 8-way\\
Compulsory & 0.0007747 &  0.00077112 & 0.00077193 & 0.00077652\\
Capacity & 0.0171105 & 0.00174216 & 0.00190593 & 0.0019116\\
Conflict & 0.0280173 & 0.03318315 & 0.00002214 & 0.00001188 \\
Total & 0.0305 & 0.0357 & 0.0027 & 0.0027
\end{matrix}
$$

$$
\begin{matrix}
 &  & L1\ config\ 2 &  & \\
Miss\ rate & 1-way & 2-way & 4-way & 8-way\\
Compulsory & 0.00038478 & 0.00038584 & 0.00039104 & 0.00038608 \\
Capacity & 0.0109989 & 0.00119756 & 0.00120896 & 0.00121392  \\
Conflict & 0.03481533 & 0.0348166  & 0.0000 & 0.0000\\
Total & 0.0363 & 0.0364 & 0.0016  & 0.0016
\end{matrix}
$$

b) For each L1 cache setup, draw a plot with the variation of the obtained compulsory, capacity,
conflict and total miss-rates for the considered set associativity ways.

c) Comment the results above

d) Write the expression that provides the mean access time as a function of the L1 cache hit
and miss
rates, the L1 cache hit
and miss
access times, and the time penalty
associated to each associativity level, as expressed in Table 1.\
 Consider a non-blocking criticalword-first load policy, where the bus occupancy rate has a lower impact in the performance of the
cache.

In [32]:
from math import log2


def mean_acess_time (miss_rate, num_of_ways):

    hit_time = 2 * (0.7 + 0.35 * log2(num_of_ways))

    miss_penalty = 140

    return hit_time + miss_rate * miss_penalty


e) Evaluate the mean access time of each configuration, considering the obtained miss-rates and the
time penalty associated to each associativity level.\
Evaluate the resulting cost function, as defined
in Eq. 1 (including the frame memory).\
Fill the following table with the obtained data:

In [35]:
print("####### Access Time #######")
print("L1 config 1")
print(mean_acess_time(0.0305,1))
print(mean_acess_time(0.0357,2))
print(mean_acess_time(0.0027,4))
print(mean_acess_time(0.0027,8))
print("L1 config 2")
print(mean_acess_time(0.0363,1))
print(mean_acess_time(0.0364,2))
print(mean_acess_time(0.0016,4))
print(mean_acess_time(0.0016,8))
print("\n####### Price #######")
print("Config 1 and 2 price")
print(calculate_price(pow(2,10), PRICE_PER_MBYTE_L1))
print("\n####### Cost Function #######")
print("L1 config 1")
print( mean_acess_time(0.0305,1) *calculate_price(pow(2,10), PRICE_PER_MBYTE_L1))
print( mean_acess_time(0.0357,2) *calculate_price(pow(2,10), PRICE_PER_MBYTE_L1))
print( mean_acess_time(0.0027,4) *calculate_price(pow(2,10), PRICE_PER_MBYTE_L1))
print( mean_acess_time(0.0027,8) *calculate_price(pow(2,10), PRICE_PER_MBYTE_L1))
print("L1 config 2")
print(mean_acess_time(0.0363,1)*calculate_price(pow(2,10), PRICE_PER_MBYTE_L1) )
print(mean_acess_time(0.0364,2)*calculate_price(pow(2,10), PRICE_PER_MBYTE_L1) )
print(mean_acess_time(0.0016,4)*calculate_price(pow(2,10), PRICE_PER_MBYTE_L1) )
print(mean_acess_time(0.0016,8)*calculate_price(pow(2,10), PRICE_PER_MBYTE_L1) )



####### Access Time #######
L1 config 1
5.67
7.098
3.178
3.8779999999999997
L1 config 2
6.481999999999999
7.196
3.024
3.7239999999999998

####### Price #######
L1 price
0.009765625
L2 price
0.009765625

####### Cost Function #######
L1 config 1
0.05537109375
0.06931640625
0.03103515625
0.037871093749999994
L1 config 2
0.06330078125
0.0702734375
0.029531250000000002
0.036367187499999995


$$
\begin{matrix}
 &  & L1\ config\ 1 &  & \\
 & 1-way & 2-way & 4-way & 8-way\\
Miss\ rate &   0.0305 & 0.0357 & 0.0027 & 0.0027 \\
Acess\ time & 5.67 & 7.098 & 3.178 & 3.8779999\\
Price & & 0.009765625  & & \\
Cost\ function &  &  &  & 
\end{matrix}
$$
$$
\begin{matrix}
 &  & L1\ config\ 2 &  & \\
 & 1-way & 2-way & 4-way & 8-way\\
Miss\ rate &  0.0363 & 0.0364 & 0.0016  & 0.0016 \\
Acess\ time & 6.481999 & 7.196  & 3.024 & 3.723999\\
Price & & 0.009765625 & &  \\
Cost\ function &  &  &  & 
\end{matrix}
$$

f) Draw conclusions:

#### 2.3.4 Cache L1: write policy

a) By analyzing the sequence of memory accesses generated by the motion estimation algorithm (see
Fig. 3), select the best setup for the cache writing-policy: write-back versus write-through, writeallocate versus write-not-allocate.\
Justify. (Note that the number of writes is much smaller than
the number of reads.)

#### 2.3.5 Cache L1: final selection

a) By considering the obtained results, select the L1 cache setup that offers the best compromise
between the cost of the device and the resulting average access time.

$$
\begin{matrix}
 & L1\ config\ 1\\
Cache\ dimension & \\
Block\ size & \\
Associativity & \\
Write\ policy & \\
Miss\ rate & \\
Acess\ time & \\
Price & \\
Cost\ function & 
\end{matrix}
$$

### 2.4 Cache L2

#### 2.4.1 Cache L2: dimension

a) Considering the maximum cost of the whole memory hierarchy, as well as the price of L1 cache
and the 128 kByte frame memory, determine the maximum storage space of L2 cache (an integer
power of 2), considering the characteristics of the available memory devices (see Table 1).

b) For the obtained maximum storage space for L2 cache, adopting a direct mapping configuration,
use dineroIV simulator to evaluate the resulting average data miss-rate considering the following
block sizes: (1 × L1_block), (2 × L1_block), (4 × L1_block) and (8 × L1_block).\
Fill the following table with the obtained data:

c) Plot the variation of the miss-rate with the size of the block. 

d) From the obtained results, select the block size that offers the best trade-off between the resulting
average miss-rate and the time penalty associated with each data fetch from the primary memory.\
Justify.

#### 2.4.2 Cache L2

a) Evaluate the compulsory, capacity, conflict and total miss-rates for the direct-mapped L2 data
cache.\
Fill the following table with the obtained data:


$$
\begin{matrix}
& Miss Rate\\
Compulsory &   \\
Capacity   & \\
Conflict   & \\
Total      &
\end{matrix}
$$

b) Plot the variation of the obtained compulsory, capacity, conflict and total miss-rate.

c) Write the expression which provides the mean access time as a function of the L1 and L2 cache
hit and miss  rates, L1 and L2 cache hit and miss  access
times, and the time penalty, as expressed in table 1.

d) Evaluate the mean access time provided by the chosen configuration, considering the obtained
miss-rate and the time penalty. Evaluate the resulting cost function, as defined in Eq. 1.\
Fill the following table with the obtained data:

$$
\begin{matrix}
 Miss Rate &\\
Acess time &   \\
Price   & \\
Cost function   & \\
\end{matrix}
$$

### 2.5 Memory Hierarchy Configuration

a) By considering the obtained results, fill the following table with the selected characteristics for L1
and L2 cache memories, as well as the corresponding performance results of the overall memory
hierarchy.

$$
\begin{matrix}
 & Cache\ L1 & Cache\ L2 & Frame\ Memory\\
Dimension\ ( Bytes) &  &  & 128\ *\ 1024\\
Block\ size\ ( Bytes) &  &  & -\\
Associativity &  &  & -\\
Write\ policy &  &  & -\\
Local\ Miss\ rate\ ( \%) &  &  & -\\
Price\ ( € ) &  &  & \\
Global\ Miss\ rate\ ( \%) &  &  & \\
Global\ access\ rate\ ( ns) &  &  & \\
Total\ Price\ ( € ) &  &  & \\
Cost\ function\ ( € \ *\ ns) &  &  & 
\end{matrix}
$$